## Write final GS MAP / CIs from top and bottom to csv
Author: Robert Ietswaart  
Date: 20220118  
License: BSD2.  
Load modules j3dl and activate virtual environment using j4RNAdecay on O2.  
Python v3.7.4

Source: `Timescale_Bayes_20211026_vis_fit.ipynb`  
For Subcellular Timelapse seq project. 

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import logging
import argparse
import fit

from __init__ import default_logger_format, default_date_format

In [2]:
# def main():
np.random.seed(12345)

parser = argparse.ArgumentParser(
    description='Write final GS MAP / CIs from top and bottom to csv.')

args = parser.parse_args("")#EDIT: added "" as argument to run in ipynb instead of .py script

In [3]:
outpath = os.path.join('/n','groups','churchman','ri23','bseq','GS20220118')

# Add a logger specific to the project and processing stage
logger = logging.getLogger('GS_final')
log_file = os.path.join(outpath,'LogErr', 'GS_20220118_MAP_CIs.log')
formatter = logging.Formatter(default_logger_format,
                              datefmt=default_date_format)
log_handler = logging.FileHandler(log_file)
log_handler.setFormatter(formatter)
logger.addHandler(log_handler)

organisms = ['m','h']
fracs = ['chr', 'nuc', 'nucpl', 'cyto', 'poly', 'tot']
reps = ['G','H','R','S','T','U']
org_reps = {'m': ['G','H','R','S'], 'h': ['T', 'U']}
red_r = {'G': 'G_R', 'H': 'H_S', 'R': 'G_R', 'S': 'H_S', 'T': 'T', 'U': 'U'}
red_reps = ['G_R', 'H_S', 'T', 'U']
time_id = [str(i) for i in range(1,6)]
background_id = {r: '1' for r in reps}
# time_mins = [0, 15, 30, 60, 120]


TC_TYPES = ['top1000','bottom500']#BM model turnover method to estimate TC
TC_from_jit = {1: 'top1000', 0: 'bottom500'}
OUT_TYPES = ['.0.05.quantile', '.Mean', '.MAP', '.0.95.quantile'] #GS specific

GS = dict()         #GRAND-SLAM
TB = dict()         #TopBottom genes
SUFFICES = {o: dict() for o in organisms}
SUFFICES['m'] = {'top1000': [''], 'bottom500': ['', '_below']}
SUFFICES['h'] = {'top1000': ['genes.turnover'], 'bottom500': ['', '_below']}


for o in organisms:
    GS[o] = dict()
    TB[o] = dict()
    
    if o == 'm':
        path_gs = os.path.join('/n','groups','churchman','ri23','bseq','GS20210506')

    elif o == 'h':
        path_gs = os.path.join('/n','groups','churchman','ri23','bseq','GS20210713_human')

    
    for r in reps:
        for fr in fracs:
            for tc in TC_TYPES:
                filename_gs = r + '_' + fr + '_noMT_' + tc + '.csv'
                if os.path.exists(os.path.join(path_gs, filename_gs)):
                    GS[o][red_r[r]+fr+tc]= pd.read_csv(os.path.join(path_gs, filename_gs) ,index_col=0)
                    GS[o][red_r[r]+fr+tc].sort_values(by='Gene',inplace=True, ignore_index=True)
                    
                    for suffix in SUFFICES[o][tc]:
                        filename = r + '1-' + fr + '_' + tc + suffix + '.MAPs.txt'
                        if tc == 'top1000':
                            tb_key = red_r[r] + fr + tc
                        if tc == 'bottom500':
                            tb_key = red_r[r] + fr + tc + suffix
                        TB[o][tb_key]= pd.read_csv(os.path.join(path_gs, filename), 
                                                     sep='\t', header=None, 
                                                     names=['ENS_ID', 'Symbol', 'MAP'])

## Process GS top and bottom to get final MAP / CIs

In [6]:
for o in organisms:
    for fr in fracs: 
        for r in org_reps[o]:
            rr = red_r[r]
            tc0 = 'top1000' 
            if r+background_id[r]+'.MAP' in GS[o][rr+fr+tc0].columns:
                logger.info('%s, %s, %s' % (o, r, fr))
                N_genes = len(GS[o][rr+fr+tc0])
                GS[o][rr+fr+'final'] = dict()
                for col in ['Gene', 'Symbol']:
                    GS[o][rr+fr+'final'][col] = []
                for t in time_id:
                    for ot in OUT_TYPES:
                        col = r+t+ot
                        GS[o][rr+fr+'final'][col] = []
#                 print(r+fr)
#                 print(GS[o][rr+fr+'final'].keys(),'\n')
                
                for g_idx in GS[o][rr+fr+tc0].index:
                    if (g_idx % 100) == 0:
                        logger.info('%d / %d' % (g_idx, N_genes))
                    gene = GS[o][rr+fr+tc0]['Gene'][g_idx]
                    GS[o][rr+fr+'final']['Gene'].append(gene)
                    GS[o][rr+fr+'final']['Symbol'].append(GS[o][rr+fr+tc0]['Symbol'][g_idx])
                    
                    #get NTR timeseries from top1000 and bottom500
                    TC_TYPES_gene = fit.get_tc_types_for_gene_jit(gene, rr, fr, GS[o].keys(), TB[o])
                    
                    NTR = dict()
                    for ot in OUT_TYPES:
                        gs_times = [r+t+ot for t in time_id]                            
                        for i in TC_TYPES_gene:
                            tc = TC_from_jit[i]                      
                            if not ot in NTR.keys():
                                NTR[ot] = np.asarray(GS[o][rr+fr+tc].loc[g_idx,gs_times], 
                                                     dtype='float64')
                            else:
                                if ot in ['.Mean','.MAP']:
                                    NTR[ot] = (NTR[ot] + np.asarray(GS[o][rr+fr+tc].loc[g_idx, gs_times], 
                                                                    dtype='float64')) / 2
                                elif ot == '.0.05.quantile':
                                    NTR[ot] = np.minimum(NTR[ot],
                                                         np.asarray(GS[o][rr+fr+tc].loc[g_idx, gs_times], 
                                                                    dtype='float64'))
                                elif ot == '.0.95.quantile':
                                    NTR[ot] = np.maximum(NTR[ot],
                                                         np.asarray(GS[o][rr+fr+tc].loc[g_idx, gs_times], 
                                                                    dtype='float64'))                                    
                        for t in time_id:
                            col = r+t+ot
                            GS[o][rr+fr+'final'][col].append(NTR[ot][(int(t)-1)])

                GS_final_df = pd.DataFrame(GS[o][rr+fr+'final'])
                filename = 'GS20220118_' + r + '_' + fr + '.tsv'
                GS_final_df.to_csv(os.path.join(outpath, filename), sep='\t',index=False)
                
logger.info('end')

INFO: [2022-01-18 16:14:47] GS_final - m, R, chr
INFO: [2022-01-18 16:14:47] GS_final - 0 / 15197
INFO: [2022-01-18 16:14:48] GS_final - 100 / 15197
INFO: [2022-01-18 16:14:49] GS_final - 200 / 15197
INFO: [2022-01-18 16:14:50] GS_final - 300 / 15197
INFO: [2022-01-18 16:14:51] GS_final - 400 / 15197
INFO: [2022-01-18 16:14:52] GS_final - 500 / 15197
INFO: [2022-01-18 16:14:53] GS_final - 600 / 15197
INFO: [2022-01-18 16:14:53] GS_final - 700 / 15197
INFO: [2022-01-18 16:14:54] GS_final - 800 / 15197
INFO: [2022-01-18 16:14:55] GS_final - 900 / 15197
INFO: [2022-01-18 16:14:56] GS_final - 1000 / 15197
INFO: [2022-01-18 16:14:57] GS_final - 1100 / 15197
INFO: [2022-01-18 16:14:58] GS_final - 1200 / 15197
INFO: [2022-01-18 16:14:59] GS_final - 1300 / 15197
INFO: [2022-01-18 16:14:59] GS_final - 1400 / 15197
INFO: [2022-01-18 16:15:00] GS_final - 1500 / 15197
INFO: [2022-01-18 16:15:01] GS_final - 1600 / 15197
INFO: [2022-01-18 16:15:02] GS_final - 1700 / 15197
INFO: [2022-01-18 16:15:03]

INFO: [2022-01-18 16:17:03] GS_final - 300 / 15019
INFO: [2022-01-18 16:17:04] GS_final - 400 / 15019
INFO: [2022-01-18 16:17:05] GS_final - 500 / 15019
INFO: [2022-01-18 16:17:06] GS_final - 600 / 15019
INFO: [2022-01-18 16:17:07] GS_final - 700 / 15019
INFO: [2022-01-18 16:17:07] GS_final - 800 / 15019
INFO: [2022-01-18 16:17:08] GS_final - 900 / 15019
INFO: [2022-01-18 16:17:09] GS_final - 1000 / 15019
INFO: [2022-01-18 16:17:10] GS_final - 1100 / 15019
INFO: [2022-01-18 16:17:11] GS_final - 1200 / 15019
INFO: [2022-01-18 16:17:12] GS_final - 1300 / 15019
INFO: [2022-01-18 16:17:13] GS_final - 1400 / 15019
INFO: [2022-01-18 16:17:13] GS_final - 1500 / 15019
INFO: [2022-01-18 16:17:14] GS_final - 1600 / 15019
INFO: [2022-01-18 16:17:15] GS_final - 1700 / 15019
INFO: [2022-01-18 16:17:16] GS_final - 1800 / 15019
INFO: [2022-01-18 16:17:17] GS_final - 1900 / 15019
INFO: [2022-01-18 16:17:18] GS_final - 2000 / 15019
INFO: [2022-01-18 16:17:19] GS_final - 2100 / 15019
INFO: [2022-01-18 1

INFO: [2022-01-18 16:19:18] GS_final - 800 / 14567
INFO: [2022-01-18 16:19:19] GS_final - 900 / 14567
INFO: [2022-01-18 16:19:19] GS_final - 1000 / 14567
INFO: [2022-01-18 16:19:20] GS_final - 1100 / 14567
INFO: [2022-01-18 16:19:21] GS_final - 1200 / 14567
INFO: [2022-01-18 16:19:22] GS_final - 1300 / 14567
INFO: [2022-01-18 16:19:23] GS_final - 1400 / 14567
INFO: [2022-01-18 16:19:24] GS_final - 1500 / 14567
INFO: [2022-01-18 16:19:25] GS_final - 1600 / 14567
INFO: [2022-01-18 16:19:25] GS_final - 1700 / 14567
INFO: [2022-01-18 16:19:26] GS_final - 1800 / 14567
INFO: [2022-01-18 16:19:27] GS_final - 1900 / 14567
INFO: [2022-01-18 16:19:28] GS_final - 2000 / 14567
INFO: [2022-01-18 16:19:29] GS_final - 2100 / 14567
INFO: [2022-01-18 16:19:30] GS_final - 2200 / 14567
INFO: [2022-01-18 16:19:31] GS_final - 2300 / 14567
INFO: [2022-01-18 16:19:31] GS_final - 2400 / 14567
INFO: [2022-01-18 16:19:32] GS_final - 2500 / 14567
INFO: [2022-01-18 16:19:33] GS_final - 2600 / 14567
INFO: [2022-01

INFO: [2022-01-18 16:21:32] GS_final - 1800 / 14655
INFO: [2022-01-18 16:21:33] GS_final - 1900 / 14655
INFO: [2022-01-18 16:21:34] GS_final - 2000 / 14655
INFO: [2022-01-18 16:21:35] GS_final - 2100 / 14655
INFO: [2022-01-18 16:21:36] GS_final - 2200 / 14655
INFO: [2022-01-18 16:21:36] GS_final - 2300 / 14655
INFO: [2022-01-18 16:21:37] GS_final - 2400 / 14655
INFO: [2022-01-18 16:21:38] GS_final - 2500 / 14655
INFO: [2022-01-18 16:21:39] GS_final - 2600 / 14655
INFO: [2022-01-18 16:21:40] GS_final - 2700 / 14655
INFO: [2022-01-18 16:21:41] GS_final - 2800 / 14655
INFO: [2022-01-18 16:21:41] GS_final - 2900 / 14655
INFO: [2022-01-18 16:21:42] GS_final - 3000 / 14655
INFO: [2022-01-18 16:21:43] GS_final - 3100 / 14655
INFO: [2022-01-18 16:21:44] GS_final - 3200 / 14655
INFO: [2022-01-18 16:21:45] GS_final - 3300 / 14655
INFO: [2022-01-18 16:21:46] GS_final - 3400 / 14655
INFO: [2022-01-18 16:21:47] GS_final - 3500 / 14655
INFO: [2022-01-18 16:21:47] GS_final - 3600 / 14655
INFO: [2022-

INFO: [2022-01-18 16:23:46] GS_final - 2700 / 14149
INFO: [2022-01-18 16:23:47] GS_final - 2800 / 14149
INFO: [2022-01-18 16:23:48] GS_final - 2900 / 14149
INFO: [2022-01-18 16:23:49] GS_final - 3000 / 14149
INFO: [2022-01-18 16:23:50] GS_final - 3100 / 14149
INFO: [2022-01-18 16:23:51] GS_final - 3200 / 14149
INFO: [2022-01-18 16:23:51] GS_final - 3300 / 14149
INFO: [2022-01-18 16:23:52] GS_final - 3400 / 14149
INFO: [2022-01-18 16:23:53] GS_final - 3500 / 14149
INFO: [2022-01-18 16:23:54] GS_final - 3600 / 14149
INFO: [2022-01-18 16:23:55] GS_final - 3700 / 14149
INFO: [2022-01-18 16:23:56] GS_final - 3800 / 14149
INFO: [2022-01-18 16:23:56] GS_final - 3900 / 14149
INFO: [2022-01-18 16:23:57] GS_final - 4000 / 14149
INFO: [2022-01-18 16:23:58] GS_final - 4100 / 14149
INFO: [2022-01-18 16:23:59] GS_final - 4200 / 14149
INFO: [2022-01-18 16:24:00] GS_final - 4300 / 14149
INFO: [2022-01-18 16:24:01] GS_final - 4400 / 14149
INFO: [2022-01-18 16:24:02] GS_final - 4500 / 14149
INFO: [2022-

...

INFO: [2022-01-18 16:55:37] GS_final - 12400 / 15247
INFO: [2022-01-18 16:55:38] GS_final - 12500 / 15247
INFO: [2022-01-18 16:55:39] GS_final - 12600 / 15247
INFO: [2022-01-18 16:55:40] GS_final - 12700 / 15247
INFO: [2022-01-18 16:55:40] GS_final - 12800 / 15247
INFO: [2022-01-18 16:55:41] GS_final - 12900 / 15247
INFO: [2022-01-18 16:55:42] GS_final - 13000 / 15247
INFO: [2022-01-18 16:55:43] GS_final - 13100 / 15247
INFO: [2022-01-18 16:55:44] GS_final - 13200 / 15247
INFO: [2022-01-18 16:55:44] GS_final - 13300 / 15247
INFO: [2022-01-18 16:55:45] GS_final - 13400 / 15247
INFO: [2022-01-18 16:55:46] GS_final - 13500 / 15247
INFO: [2022-01-18 16:55:47] GS_final - 13600 / 15247
INFO: [2022-01-18 16:55:48] GS_final - 13700 / 15247
INFO: [2022-01-18 16:55:49] GS_final - 13800 / 15247
INFO: [2022-01-18 16:55:49] GS_final - 13900 / 15247
INFO: [2022-01-18 16:55:50] GS_final - 14000 / 15247
INFO: [2022-01-18 16:55:51] GS_final - 14100 / 15247
INFO: [2022-01-18 16:55:52] GS_final - 14200 /

INFO: [2022-01-18 16:57:43] GS_final - 12700 / 15056
INFO: [2022-01-18 16:57:44] GS_final - 12800 / 15056
INFO: [2022-01-18 16:57:45] GS_final - 12900 / 15056
INFO: [2022-01-18 16:57:46] GS_final - 13000 / 15056
INFO: [2022-01-18 16:57:47] GS_final - 13100 / 15056
INFO: [2022-01-18 16:57:48] GS_final - 13200 / 15056
INFO: [2022-01-18 16:57:48] GS_final - 13300 / 15056
INFO: [2022-01-18 16:57:49] GS_final - 13400 / 15056
INFO: [2022-01-18 16:57:50] GS_final - 13500 / 15056
INFO: [2022-01-18 16:57:51] GS_final - 13600 / 15056
INFO: [2022-01-18 16:57:52] GS_final - 13700 / 15056
INFO: [2022-01-18 16:57:53] GS_final - 13800 / 15056
INFO: [2022-01-18 16:57:53] GS_final - 13900 / 15056
INFO: [2022-01-18 16:57:54] GS_final - 14000 / 15056
INFO: [2022-01-18 16:57:55] GS_final - 14100 / 15056
INFO: [2022-01-18 16:57:56] GS_final - 14200 / 15056
INFO: [2022-01-18 16:57:57] GS_final - 14300 / 15056
INFO: [2022-01-18 16:57:58] GS_final - 14400 / 15056
INFO: [2022-01-18 16:57:59] GS_final - 14500 /

INFO: [2022-01-18 16:59:50] GS_final - 13200 / 14947
INFO: [2022-01-18 16:59:51] GS_final - 13300 / 14947
INFO: [2022-01-18 16:59:51] GS_final - 13400 / 14947
INFO: [2022-01-18 16:59:52] GS_final - 13500 / 14947
INFO: [2022-01-18 16:59:53] GS_final - 13600 / 14947
INFO: [2022-01-18 16:59:54] GS_final - 13700 / 14947
INFO: [2022-01-18 16:59:55] GS_final - 13800 / 14947
INFO: [2022-01-18 16:59:56] GS_final - 13900 / 14947
INFO: [2022-01-18 16:59:56] GS_final - 14000 / 14947
INFO: [2022-01-18 16:59:57] GS_final - 14100 / 14947
INFO: [2022-01-18 16:59:58] GS_final - 14200 / 14947
INFO: [2022-01-18 16:59:59] GS_final - 14300 / 14947
INFO: [2022-01-18 17:00:00] GS_final - 14400 / 14947
INFO: [2022-01-18 17:00:01] GS_final - 14500 / 14947
INFO: [2022-01-18 17:00:02] GS_final - 14600 / 14947
INFO: [2022-01-18 17:00:02] GS_final - 14700 / 14947
INFO: [2022-01-18 17:00:03] GS_final - 14800 / 14947
INFO: [2022-01-18 17:00:04] GS_final - 14900 / 14947
INFO: [2022-01-18 17:00:05] GS_final - h, U, p

INFO: [2022-01-18 17:01:54] GS_final - 13800 / 14902
INFO: [2022-01-18 17:01:55] GS_final - 13900 / 14902
INFO: [2022-01-18 17:01:56] GS_final - 14000 / 14902
INFO: [2022-01-18 17:01:57] GS_final - 14100 / 14902
INFO: [2022-01-18 17:01:58] GS_final - 14200 / 14902
INFO: [2022-01-18 17:01:59] GS_final - 14300 / 14902
INFO: [2022-01-18 17:01:59] GS_final - 14400 / 14902
INFO: [2022-01-18 17:02:00] GS_final - 14500 / 14902
INFO: [2022-01-18 17:02:01] GS_final - 14600 / 14902
INFO: [2022-01-18 17:02:02] GS_final - 14700 / 14902
INFO: [2022-01-18 17:02:03] GS_final - 14800 / 14902
INFO: [2022-01-18 17:02:04] GS_final - 14900 / 14902
INFO: [2022-01-18 17:02:04] GS_final - h, T, tot
INFO: [2022-01-18 17:02:04] GS_final - 0 / 14435
INFO: [2022-01-18 17:02:05] GS_final - 100 / 14435
INFO: [2022-01-18 17:02:06] GS_final - 200 / 14435
INFO: [2022-01-18 17:02:07] GS_final - 300 / 14435
INFO: [2022-01-18 17:02:07] GS_final - 400 / 14435
INFO: [2022-01-18 17:02:08] GS_final - 500 / 14435
INFO: [2022

INFO: [2022-01-18 17:04:06] GS_final - 14400 / 14435
INFO: [2022-01-18 17:04:06] GS_final - h, U, tot
INFO: [2022-01-18 17:04:06] GS_final - 0 / 14262
INFO: [2022-01-18 17:04:07] GS_final - 100 / 14262
INFO: [2022-01-18 17:04:08] GS_final - 200 / 14262
INFO: [2022-01-18 17:04:09] GS_final - 300 / 14262
INFO: [2022-01-18 17:04:10] GS_final - 400 / 14262
INFO: [2022-01-18 17:04:11] GS_final - 500 / 14262
INFO: [2022-01-18 17:04:11] GS_final - 600 / 14262
INFO: [2022-01-18 17:04:12] GS_final - 700 / 14262
INFO: [2022-01-18 17:04:13] GS_final - 800 / 14262
INFO: [2022-01-18 17:04:14] GS_final - 900 / 14262
INFO: [2022-01-18 17:04:15] GS_final - 1000 / 14262
INFO: [2022-01-18 17:04:15] GS_final - 1100 / 14262
INFO: [2022-01-18 17:04:16] GS_final - 1200 / 14262
INFO: [2022-01-18 17:04:17] GS_final - 1300 / 14262
INFO: [2022-01-18 17:04:18] GS_final - 1400 / 14262
INFO: [2022-01-18 17:04:19] GS_final - 1500 / 14262
INFO: [2022-01-18 17:04:20] GS_final - 1600 / 14262
INFO: [2022-01-18 17:04:20

In [21]:
!pip freeze

asteval==0.9.23
attrs==19.3.0
backcall==0.1.0
bleach==3.1.4
certifi==2021.5.30
charset-normalizer==2.0.1
cycler==0.10.0
decorator==4.4.2
defusedxml==0.6.0
docopt==0.6.2
entrypoints==0.3
future==0.18.2
goatools==1.1.6
gtfparse==1.2.1
idna==3.2
importlib-metadata==1.5.2
ipykernel==5.2.0
ipython==7.13.0
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.16.0
Jinja2==2.11.1
joblib==1.1.0
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.2
jupyter-console==6.1.0
jupyter-core==4.6.3
kiwisolver==1.1.0
llvmlite==0.36.0
lmfit==1.0.2
MarkupSafe==1.1.1
matplotlib==3.2.1
mistune==0.8.4
mpmath==1.2.1
nbconvert==5.6.1
nbformat==5.0.4
networkx==2.4
notebook==6.0.3
numba==0.53.1
numba-scipy==0.3.0
numpy==1.16.5
pandas==1.0.3
pandocfilters==1.4.2
parso==0.6.2
patsy==0.5.2
pexpect==4.8.0
pickleshare==0.7.5
Pillow==7.0.0
prometheus-client==0.7.1
prompt-toolkit==3.0.4
ptyprocess==0.6.0
pydot==1.4.2
Pygments==2.6.1
pyparsing==2.4.6
pyrsistent==0.16.0
python-dateutil==2.8.1
pytz==2019.3
pyzmq==19.0.0
qtco